To speed the execution procces, Colaboraty has an option to use GPU. This option is selected by default.
First of all we need to install two packages to show better the options of UFOD.

In [1]:
!pip3 install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import os

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


# Workflow of the UFOD framework

The workflow of UFOD, depicted in the following image, captures all the necessary steps to train several object detection models and select the best one. Such a workflow can be summarised as follows. First of all, the user selects the dataset of images and some configuration parameters (mainly, the algorithms that will be trained and the frameworks or libraries that provide them). Subsequently, UFOD splits the dataset into a training set and a testing set. The training set is employed to construct several object detection models, and the best of those models is selected based on their performance on the testing set. The output of the framework is the best model, and an application, in the form of a Jupyter notebook, to employ such a model. Apart from the first step --- that is, the selection of the models to be trained --- the rest of the process is conducted automatically by UFOD without any user intervention. A more detailed explanation of the main features of this framework can be found in our [draft paper](https://github.com/ManuGar/UFOD/blob/master/draft.pdf).

<img src="https://github.com/ManuGar/UFOD/raw/master/images/DiagramUFOD.png" width="900" >

# How to use the framework

To begin, we have to import all the classes that we will need to be able to use our framework.

In [2]:
import taskLauncher
# import trainModel
import factoryModel

### Configuring the dataset path

The dataset needs to be annotated using the Pascal VOC format. An application that produces such a format is [LabelImg](https://github.com/tzutalin/labelImg). The folder containing the dataset must contain two folders:
- JPEGImages folder: containing the images. 
- Annotations folder: containing the annotations using the Pascal format. 

This dataset will be later split. If the user wants to provide a explicit split, the organisation must be as follows:
- train
  - JPEGImages: folder containing the images of the training set. 
  - Annotations: folder containing the annotations of the training set. 
- test
  - JPEGImages: folder containing the images of the testing set. 
  - Annotations: folder containing the annotations of the testing set.
 

A dataset with this organization is provided in the [fruits folder](https://github.com/ManuGar/UFOD/tree/master/fruit) or [kangaro folder](https://www.dropbox.com/s/q19cl3agw8qt8e5/kangaroo.zip).


In [ ]:
!wget "https://www.dropbox.com/s/q19cl3agw8qt8e5/kangaroo.zip?dl=0" -O kangaroo.zip
!unzip kangaroo.zip

--2020-01-14 19:47:11--  https://www.dropbox.com/s/q19cl3agw8qt8e5/kangaroo.zip?dl=0
Resolviendo www.dropbox.com (www.dropbox.com)... 162.125.68.1, 2620:100:6024:1::a27d:4401
Conectando con www.dropbox.com (www.dropbox.com)[162.125.68.1]:443... conectado.
Petición HTTP enviada, esperando respuesta... 301 Moved Permanently
Ubicación: /s/raw/q19cl3agw8qt8e5/kangaroo.zip [siguiente]
--2020-01-14 19:47:11--  https://www.dropbox.com/s/raw/q19cl3agw8qt8e5/kangaroo.zip
Reutilizando la conexión con www.dropbox.com:443.
Petición HTTP enviada, esperando respuesta... 302 Found
Ubicación: https://uc1982a5e177b1990c061e21d0df.dl.dropboxusercontent.com/cd/0/inline/AwI0s1howYz6Uld38o6y1bKhNhs4sHddDPCghAGbrUmRRAeG0xoUd-FOIvXPWzu3bMdTj-wHcbLkbOqaitDsgNVQdpcxz-vARvjRWAqwrjsjZFcmIkHYoHCLUN2ep_1cA_M/file# [siguiente]
--2020-01-14 19:47:12--  https://uc1982a5e177b1990c061e21d0df.dl.dropboxusercontent.com/cd/0/inline/AwI0s1howYz6Uld38o6y1bKhNhs4sHddDPCghAGbrUmRRAeG0xoUd-FOIvXPWzu3bMdTj-wHcbLkbOqaitDsgNVQdpc

### Preparing the environment

UFOD training process is configured by means a list of parameters, that must contain the following information:

- "dataset": path to the dataset of images. 
- "dataset_name": a name to the dataset.
- "exec": the execution mode. There are two execution modes:
  - "local". The training process is run locally. In this option it is necessary to provide the number of available gpus with the "ngpus" parameter. 
  - "slurm". The training process is run in a cluster with SLURM. There are several parameters to configure in this option, such as the execution time ("time" parameter), the partition ("partition" parameter), the gres ("gres" parameter), the memory ("mem" parameter), or the number of gpus  ("ngpus" parameter).
- "frameworks": the list of models to train. Currently, the following options are available:
  - ["Mxnet","ssdVgg16"]: SSD with the VGG16 backbone of the MXNet framework. 
  - ["Mxnet","ssdVgg16_512"]: SSD with the VGG16 backbone and image size of 512 of the MXNet framework. 
  - ["Mxnet","ssdResnet"]: SSD with the Resnet backbone of the MXNet framework.
  - ["Mxnet","ssdMobilenet"]: SSD with the mobilenet backbone of the MXNet framework.
  - ["Rcnn","mask-rcnn"]: Mask-RCNN algorithm using a Keras library. 
  - ["Darknet","yolo"]: YOLO with the Darknet framework.
  - ["Darknet","tinyYolo"]: TinyYOLO with the Darknet framework.
  - ["Retinanet","retinanet"]: RetinaNet algorithm using a Keras library. 
  - ["Tensorflow", "ssdInception"]: SSD with the inception backbone using the Tensorflow framework.
  - ["Tensorflow", "fasterRcnnResnet"]: Faster RCNN with the inception backbone using the Tensorflow framework.
  - ["Tensorflow", "rfcnResnet"]: RFCN with the inception backbone using the Tensorflow framework.
  - ["Tensorflow", "maskRcnnInception"]: Mask RCNN with the inception backbone using the Tensorflow framework.

In [ ]:
dataset_path = widgets.Text(
       value='./kangaroo',
       description='Dataset path', )
dataset_name = widgets.Text(
       value='kangaroo',
       description='Dataset name', )
box = widgets.VBox([dataset_path, dataset_name])
box

In [22]:
MxnetVgg16 = widgets.Checkbox(
           description='Mxnet-SsdVgg16',value=True)
MxnetVgg16_512 = widgets.Checkbox(
           description='Mxnet-SsdVgg16_512',value=True)
MxnetResnet = widgets.Checkbox(
           description='Mxnet-SsdResnet',value=True)
MxnetMobilenet = widgets.Checkbox(
           description='Mxnet-SsdMobilenet',value=True)
Mrcnn = widgets.Checkbox(
           description='Mask-RCNN',value=True)
Retinanet = widgets.Checkbox(
           description='Retinanet',value=True)
DarknetYolo = widgets.Checkbox(
           description='Darknet-Yolo v3',value=True)
DarknetTinyYolo = widgets.Checkbox(
           description='Darknet-Tiny Yolo v3',value=True)
TensorflowSsdInception = widgets.Checkbox(
           description='Tensorflow-SsdInception',value=True)
TensorflowfasterRcnnResnet = widgets.Checkbox(
           description='Tensorflow-FasterRCNNRestnet',value=True)
TensorflowRfcnResnet = widgets.Checkbox(
           description='Tensorflow-RFCNResnet',value=True)
TensorflowMaskRcnnInception = widgets.Checkbox(
           description='Tensorflow-MaskRCNNInception',value=True)


box1 = widgets.VBox([MxnetVgg16,MxnetVgg16_512,MxnetResnet,MxnetMobilenet])
box2 = widgets.VBox([Mrcnn,Retinanet,DarknetYolo,DarknetTinyYolo])
box3 = widgets.VBox([TensorflowSsdInception,
                    TensorflowfasterRcnnResnet,TensorflowRfcnResnet,TensorflowMaskRcnnInception])

frameworks=[]
widgets.HBox([box1, box2,box3])


In [23]:
if(MxnetVgg16.value):
    frameworks.append(["Mxnet","ssdVgg16"])
if(MxnetVgg16_512.value):
    frameworks.append(["Mxnet","ssdVgg16_512"])
if(MxnetResnet.value):
    frameworks.append(["Mxnet","ssdResnet"])
if(MxnetMobilenet.value):
    frameworks.append(["Mxnet","ssdMobilenet"])
if(Mrcnn.value):
    frameworks.append(["Rcnn","mask-rcnn"])
if(Retinanet.value):
    frameworks.append(["Retinanet","retinanet"])
if(DarknetYolo.value):
    frameworks.append(["Darknet","yolo"])
if(DarknetTinyYolo.value):
    frameworks.append(["Darknet","tinyYolo"])
if(TensorflowSsdInception.value):
    frameworks.append(["Tensorflow","ssdInception"])
if(TensorflowfasterRcnnResnet.value):
    frameworks.append(["Tensorflow","fasterRcnnResnet"])
if(TensorflowMaskRcnnInception.value):
    frameworks.append(["Tensorflow","rfcnResnet"])
if(TensorflowMaskRcnnInception.value):
    frameworks.append(["Tensorflow","maskRcnnInception"])



type = widgets.Dropdown(
    options=['local', 'slurm'],
    value='local',
    description='Execution type:',
    disabled=False,
)

type

Dropdown(description='Execution type:', options=('local', 'slurm'), value='local')

In [24]:
n_gpus = widgets.BoundedIntText(
        value=1,
        min=1,
        max=15,
        step=1,
        description='Number GPU\'s:',
        disabled=False
    )
if type.value=="slurm":
    time = widgets.Text(value='72:00:00',description='Time (HH:MM:SS)', )
    partition = widgets.Text(value='gpu',description='Partition', )
    gres = widgets.Text(value='gpu:kepler:2',description='Gres', )
    mem = widgets.Text(value='56GB',description='Memmory', )
    box1 =widgets.VBox([time, partition,gres])
    box2 =widgets.VBox([mem, n_gpus])
    newbox = widgets.HBox([box1,box2])
elif type.value =="local":
    newbox = widgets.HBox([n_gpus])

newbox

### How to launch the training process

Once the user has prepared the dataset and fixed the training options in the previous parameters, the training instructions are generated using the following command.


In [25]:
for fram, mod in frameworks:
    file_name = os.path.join("./","scripts", "train_" + fram + "_" + mod + "_" + dataset_name.value + ".sh")
    f = open(file_name, "w")
    f.write("#!/bin/sh\n")
    if type =="slurm":
        if (fram == "Mxnet"):
            f.write("source configs_slurm/mxnet.sh\n")
        if (fram == "Rcnn"):
            f.write("source configs_slurm/maskrcnn.sh\n")
        if (fram == "Retinanet"):
            f.write("source configs_slurm/retinanet.sh\n")
        if (fram == "Tensorflow"):
            f.write("source configs_slurm/tensorflow.sh\n")
        if (fram == "Darknet"):
            f.write("source configs_slurm/yolo.sh\n")
    elif type =="local":
        if (fram == "Mxnet"):
            f.write("source configs_local/mxnet.sh\n")
        if (fram == "Rcnn"):
            f.write("source configs_local/maskrcnn.sh\n")
        if (fram == "Retinanet"):
            f.write("source configs_local/retinanet.sh\n")
        if (fram == "Tensorflow"):
            f.write("source configs_local/tensorflow.sh\n")
        if (fram == "Darknet"):
            f.write("source configs_local/yolo.sh\n")
    f.write("python3 trainModel.py -f " + fram + " -m " + mod + " -d " + dataset_path.value + " -dn " + dataset_name.value + " -ng " + str(n_gpus.value))
    if type.value == "slurm":
        print("!"+"sbatch -p " + partition + " --gres=" + gres + " --time=" + time+ " --mem=" + mem + " " + file_name )
    elif type.value =="local":
        print("!bash "+ file_name)



!bash ./scripts/train_Mxnet_ssdVgg16_kangaroo.sh
!bash ./scripts/train_Mxnet_ssdVgg16_512_kangaroo.sh
!bash ./scripts/train_Mxnet_ssdResnet_kangaroo.sh
!bash ./scripts/train_Mxnet_ssdMobilenet_kangaroo.sh
!bash ./scripts/train_Rcnn_mask-rcnn_kangaroo.sh
!bash ./scripts/train_Retinanet_retinanet_kangaroo.sh
!bash ./scripts/train_Darknet_yolo_kangaroo.sh
!bash ./scripts/train_Darknet_tinyYolo_kangaroo.sh
!bash ./scripts/train_Tensorflow_ssdInception_kangaroo.sh
!bash ./scripts/train_Tensorflow_fasterRcnnResnet_kangaroo.sh
!bash ./scripts/train_Tensorflow_rfcnResnet_kangaroo.sh
!bash ./scripts/train_Tensorflow_maskRcnnInception_kangaroo.sh


You can copy the instructions generated in the following cell and run the training process. 

In [27]:
!bash ./scripts/train_Rcnn_mask-rcnn_kangaroo.sh


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
W1211 18:17:57.888797 140280827893568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1211 18:17:57.930413 140280827893568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1211 18:17:57.959913 140280827893568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1211 18:17:57.996255 140280827893568 deprecation_wrapper